In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 256
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
        self.protein2go =  load_dict('ec2prot2go.pkl')
        self.protein2seq = load_dict('ec2prot2seq.pkl')
        self.read_ppi()
        self.protein2onehot = {}
        self.onehot_seqs()
        self.prot2emb = {}
        self.prot2embedding() 
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
    
    def prot2embedding(self):
        for key, value in self.protein2go.items():
            X_go1 =  np.zeros((1,768))
            allgos = value.split(',') 
            allgos = list(set(allgos))
            count = 0
            for  go in  allgos:
                feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
                if count + feature.shape[0] > max_go_len:
                    break
                X_go1 = np.concatenate((X_go1,feature ))    
                count += feature.shape[0]
            self.prot2emb[key] =  X_go1[1:]   
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        y = np.empty((self.batch_size))
        X_go1 = np.empty((self.batch_size, self.max_golen,768))
        X_go2 = np.empty((self.batch_size, self.max_golen,768))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
            prot1emb = self.prot2emb[p1]
            X_go1[i,:prot1emb.shape[0]] = prot1emb
            
            prot2emb = self.prot2emb[p2]
            X_go2[i,:prot2emb.shape[0]] = prot2emb
             
            
            
        return [X_go1, X_go2,  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

         
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        y = np.empty((len(list_IDs_temp)))
        X_go1 = np.empty((len(list_IDs_temp), self.max_golen,768))
        X_go2 = np.empty((len(list_IDs_temp), self.max_golen,768))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
            prot1emb = self.prot2emb[p1]
            X_go1[i,:prot1emb.shape[0]] = prot1emb
            
            prot2emb = self.prot2emb[p2]
            X_go2[i,:prot2emb.shape[0]] = prot2emb
           
        return [X_go1, X_go2,  X_seq1, X_seq2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [5]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.utils import multi_gpu_model
from keras.layers.merge import concatenate

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(4)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


 



def create_share_model():
    con_filters = 512
    left_input_go = Input(shape=(max_go_len,feature_len))
    left_input_seq = Input(shape=(max_seq_len,20))  
    
    cnn = Conv1D(512, 3, activation='relu',  padding='same')(left_input_go)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv1D(256, 3, activation='relu',  padding='same')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv1D(128, 3, activation='relu',  padding='same')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv1D(64, 3, activation='relu',  padding='same')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv1D(32, 3, activation='relu',  padding='same')(cnn)
    cnn = Dropout(0.1)(cnn)
    go_flat = GlobalAveragePooling1D()(cnn)
    
    
#     cnn = Conv1D(32, 3, activation='relu', padding='same')(left_input_seq) 
#     cnn = Dropout(0.1)(cnn)
#     cnn = Conv1D(256, 3, activation='relu',  padding='same')(cnn)
#     cnn = Dropout(0.1)(cnn)
#     cnn = Conv1D(128, 3, activation='relu',  padding='same')(cnn)
#     cnn = Dropout(0.1)(cnn)
#     cnn = Conv1D(64, 3, activation='relu',  padding='same')(cnn)
#     cnn = Dropout(0.1)(cnn)
#     cnn = Conv1D(32, 3, activation='relu',  padding='same')(cnn)
#     cnn = Dropout(0.1)(cnn)
    cnn = Bidirectional(CuDNNGRU(32, return_sequences=True))(left_input_seq)
    seq_flat = GlobalAveragePooling1D()(cnn)
    
    output = Concatenate()([go_flat,seq_flat])
    model = Model( [left_input_go, left_input_seq], output)
    return model




def build_siamese_model():
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    
    siamese_a = create_share_model()
    siamese_b = create_share_model()
    

    encoded_l = siamese_a([left_input_go, left_input_seq])
    encoded_r = siamese_b([right_input_go, right_input_seq])
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
   

    siamese_net = Model(inputs=[left_input_go, right_input_go, left_input_seq, right_input_seq],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return siamese_net


def build_model():
    con_filters = 128
    recurrent_units = 64
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
    x = inception_block(left_input_go,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(left_input_go)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x)
    left_x_go = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b, x_gru_c])
    left_x_go = Dense(256, activation='relu')(left_x_go)
     
 
    x = inception_block(right_input_go,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(right_input_go)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x)
    right_x_go = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b, x_gru_c])
    right_x_go = Dense(256, activation='relu')(right_x_go)
    
    
    
    x = inception_block(left_input_seq,con_filters)
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(left_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x)
    left_x_seq = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b, x_gru_c])
    left_x_seq = Dense(256, activation='relu')(left_x_seq)
     
 
    x = inception_block(right_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(right_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x)
    right_x_seq= Concatenate()([x_a  , x_b, x_c, x_gru_a, x_gru_b, x_gru_c])
    right_x_seq = Dense(256, activation='relu')(right_x_seq)
    
    
   
   
    x =   Concatenate()([left_x_go  , right_x_go, left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
  
     
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go, right_input_go, left_input_seq, right_input_seq], output)
#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 

In [6]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from keras.utils import multi_gpu_model

dataset_name = 'EC2'
for rep in range(3):
    n_splits = 10
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
     
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(split)

        batch_size = 64
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'CV/ec2_GoplusSeq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_acc', patience=10, verbose=0, mode='max')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_acc', mode='max', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        #  max_queue_size=16, workers=8, use_multiprocessing=True,
        hist = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
        del test_x
        del y_test
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('seq_and_go__incep_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
29/29 [==============================] - 29s 984ms/step - loss: 0.5857 - acc: 0.7117 - val_loss: 0.4152 - val_acc: 0.8073
Epoch 2/100
29/29 [==============================] - 16s 536ms/step - loss: 0.3781 - acc: 0.8335 - val_loss: 0.3438 - val_acc: 0.8281
Epoch 3/100
29/29 [==============================] - 15s 534ms/step - loss: 0.3351 - acc: 0.8421 - val_loss: 0.3115 - val_acc: 0.8333
Epoch 4/100
29/29 [==============================] - 16s 543ms/step - loss: 0.3050 - acc: 0.8599 - val_loss: 0.3790 - val_acc: 0.8281
Epoch 5/100
29/29 [==============================] - 16s 541ms/step - loss: 0.2916 - acc: 0.8648 - val_loss: 0.2868 - val_acc: 0.8594
Epoch 6/100
29/29 [==============================] - 14s 496ms/step - loss: 0.2749 - acc: 0.8718 - val_loss: 0.2850

29/29 [==============================] - 24s 818ms/step - loss: 0.5695 - acc: 0.7355 - val_loss: 0.4089 - val_acc: 0.8229
Epoch 2/100
29/29 [==============================] - 13s 444ms/step - loss: 0.3719 - acc: 0.8308 - val_loss: 0.3613 - val_acc: 0.8333
Epoch 3/100
29/29 [==============================] - 13s 433ms/step - loss: 0.3300 - acc: 0.8448 - val_loss: 0.3475 - val_acc: 0.8438
Epoch 4/100
29/29 [==============================] - 13s 454ms/step - loss: 0.3057 - acc: 0.8534 - val_loss: 0.3496 - val_acc: 0.8542
Epoch 5/100
29/29 [==============================] - 14s 492ms/step - loss: 0.2948 - acc: 0.8626 - val_loss: 0.3414 - val_acc: 0.8594
Epoch 6/100
29/29 [==============================] - 14s 468ms/step - loss: 0.2595 - acc: 0.8847 - val_loss: 0.3548 - val_acc: 0.8125
Epoch 7/100
29/29 [==============================] - 13s 444ms/step - loss: 0.2168 - acc: 0.8955 - val_loss: 0.3592 - val_acc: 0.8802
Epoch 8/100
29/29 [==============================] - 13s 456ms/step - loss

Epoch 7/100
29/29 [==============================] - 15s 502ms/step - loss: 0.2243 - acc: 0.8971 - val_loss: 0.3484 - val_acc: 0.8073
Epoch 8/100
29/29 [==============================] - 15s 506ms/step - loss: 0.2173 - acc: 0.8928 - val_loss: 0.3318 - val_acc: 0.7917
Epoch 9/100
29/29 [==============================] - 15s 523ms/step - loss: 0.2181 - acc: 0.9046 - val_loss: 0.3660 - val_acc: 0.8021
Epoch 10/100
29/29 [==============================] - 15s 518ms/step - loss: 0.1971 - acc: 0.9122 - val_loss: 0.4454 - val_acc: 0.8125
Epoch 11/100
29/29 [==============================] - 15s 523ms/step - loss: 0.1862 - acc: 0.9192 - val_loss: 0.3725 - val_acc: 0.8177
Epoch 12/100
29/29 [==============================] - 15s 519ms/step - loss: 0.1478 - acc: 0.9364 - val_loss: 0.3812 - val_acc: 0.8021
Epoch 13/100
29/29 [==============================] - 16s 536ms/step - loss: 0.1324 - acc: 0.9461 - val_loss: 0.4458 - val_acc: 0.8125
Epoch 14/100
29/29 [==============================] - 15s 

29/29 [==============================] - 16s 535ms/step - loss: 0.2322 - acc: 0.8955 - val_loss: 0.3927 - val_acc: 0.8385
Epoch 9/100
29/29 [==============================] - 15s 531ms/step - loss: 0.2001 - acc: 0.9127 - val_loss: 0.4137 - val_acc: 0.8750
Epoch 10/100
29/29 [==============================] - 15s 532ms/step - loss: 0.1874 - acc: 0.9143 - val_loss: 0.5323 - val_acc: 0.8646
Epoch 11/100
29/29 [==============================] - 17s 588ms/step - loss: 0.1789 - acc: 0.9235 - val_loss: 0.3877 - val_acc: 0.8385
Epoch 12/100
29/29 [==============================] - 26s 886ms/step - loss: 0.1414 - acc: 0.9423 - val_loss: 0.4183 - val_acc: 0.8490
Epoch 13/100
29/29 [==============================] - 21s 718ms/step - loss: 0.1238 - acc: 0.9510 - val_loss: 0.4644 - val_acc: 0.8594
Epoch 14/100
29/29 [==============================] - 15s 518ms/step - loss: 0.1358 - acc: 0.9461 - val_loss: 0.7182 - val_acc: 0.7812
Epoch 15/100
29/29 [==============================] - 15s 531ms/step 

Epoch 1/100
29/29 [==============================] - 30s 1s/step - loss: 0.6113 - acc: 0.7074 - val_loss: 0.3357 - val_acc: 0.8646
Epoch 2/100
29/29 [==============================] - 10s 360ms/step - loss: 0.3953 - acc: 0.8217 - val_loss: 0.3290 - val_acc: 0.8646
Epoch 3/100
29/29 [==============================] - 11s 374ms/step - loss: 0.3429 - acc: 0.8378 - val_loss: 0.2805 - val_acc: 0.8906
Epoch 4/100
29/29 [==============================] - 11s 370ms/step - loss: 0.3025 - acc: 0.8481 - val_loss: 0.3023 - val_acc: 0.8750
Epoch 5/100
29/29 [==============================] - 11s 370ms/step - loss: 0.2998 - acc: 0.8556 - val_loss: 0.2972 - val_acc: 0.8698
Epoch 6/100
29/29 [==============================] - 11s 369ms/step - loss: 0.2730 - acc: 0.8675 - val_loss: 0.2955 - val_acc: 0.8698
Epoch 7/100
29/29 [==============================] - 11s 377ms/step - loss: 0.2348 - acc: 0.8858 - val_loss: 0.2930 - val_acc: 0.8490
Epoch 8/100
29/29 [==============================] - 11s 391ms/st

...

Epoch 6/100
29/29 [==============================] - 17s 577ms/step - loss: 0.2677 - acc: 0.8793 - val_loss: 0.3244 - val_acc: 0.8542
Epoch 7/100
29/29 [==============================] - 13s 454ms/step - loss: 0.2427 - acc: 0.8874 - val_loss: 0.3670 - val_acc: 0.7812
Epoch 8/100
29/29 [==============================] - 13s 441ms/step - loss: 0.2319 - acc: 0.8971 - val_loss: 0.3202 - val_acc: 0.8646
Epoch 9/100
29/29 [==============================] - 17s 589ms/step - loss: 0.2190 - acc: 0.9116 - val_loss: 0.3347 - val_acc: 0.8281
Epoch 10/100
29/29 [==============================] - 16s 561ms/step - loss: 0.1882 - acc: 0.9176 - val_loss: 0.4000 - val_acc: 0.7708
Epoch 11/100
29/29 [==============================] - 16s 566ms/step - loss: 0.1963 - acc: 0.9100 - val_loss: 0.3487 - val_acc: 0.8177
Epoch 12/100
29/29 [==============================] - 15s 518ms/step - loss: 0.1807 - acc: 0.9273 - val_loss: 0.3595 - val_acc: 0.8229
Epoch 13/100
29/29 [==============================] - 16s 5

29/29 [==============================] - 14s 498ms/step - loss: 0.3329 - acc: 0.8416 - val_loss: 0.3185 - val_acc: 0.8542
Epoch 4/100
29/29 [==============================] - 15s 521ms/step - loss: 0.2918 - acc: 0.8567 - val_loss: 0.3837 - val_acc: 0.8438
Epoch 5/100
29/29 [==============================] - 15s 516ms/step - loss: 0.3006 - acc: 0.8572 - val_loss: 0.3346 - val_acc: 0.8438
Epoch 6/100
29/29 [==============================] - 15s 519ms/step - loss: 0.2618 - acc: 0.8750 - val_loss: 0.3110 - val_acc: 0.8594
Epoch 7/100
29/29 [==============================] - 15s 501ms/step - loss: 0.2436 - acc: 0.8901 - val_loss: 0.3103 - val_acc: 0.8490
Epoch 8/100
29/29 [==============================] - 13s 443ms/step - loss: 0.2244 - acc: 0.8933 - val_loss: 0.3039 - val_acc: 0.8646
Epoch 9/100
29/29 [==============================] - 11s 392ms/step - loss: 0.2004 - acc: 0.9143 - val_loss: 0.3044 - val_acc: 0.8542
Epoch 10/100
29/29 [==============================] - 11s 375ms/step - los

Epoch 13/100
29/29 [==============================] - 16s 553ms/step - loss: 0.1416 - acc: 0.9359 - val_loss: 0.4576 - val_acc: 0.8385
Epoch 14/100
29/29 [==============================] - 16s 552ms/step - loss: 0.1354 - acc: 0.9434 - val_loss: 0.5695 - val_acc: 0.7969
Epoch 15/100
29/29 [==============================] - 16s 554ms/step - loss: 0.1058 - acc: 0.9596 - val_loss: 0.5261 - val_acc: 0.8542
Epoch 16/100
29/29 [==============================] - 16s 559ms/step - loss: 0.1266 - acc: 0.9510 - val_loss: 0.4646 - val_acc: 0.8073
Epoch 17/100
29/29 [==============================] - 16s 565ms/step - loss: 0.1133 - acc: 0.9564 - val_loss: 0.5283 - val_acc: 0.7760
--------------------------

AUC: 0.978811
ACC: 0.905983
MCC : 0.813703
TPR:0.828571
FPR:0.031008
Pre:0.956044
F1:0.887755
--------------------------

Epoch 1/100
29/29 [==============================] - 57s 2s/step - loss: 0.5349 - acc: 0.7419 - val_loss: 0.3912 - val_acc: 0.8646
Epoch 2/100
29/29 [=========================

29/29 [==============================] - 57s 2s/step - loss: 0.6355 - acc: 0.6789 - val_loss: 0.4215 - val_acc: 0.8125
Epoch 2/100
29/29 [==============================] - 16s 557ms/step - loss: 0.3784 - acc: 0.8238 - val_loss: 0.3890 - val_acc: 0.8177
Epoch 3/100
29/29 [==============================] - 16s 562ms/step - loss: 0.3385 - acc: 0.8475 - val_loss: 0.3789 - val_acc: 0.8229
Epoch 4/100
29/29 [==============================] - 16s 547ms/step - loss: 0.3029 - acc: 0.8534 - val_loss: 0.4333 - val_acc: 0.8438
Epoch 5/100
29/29 [==============================] - 17s 575ms/step - loss: 0.2974 - acc: 0.8594 - val_loss: 0.4417 - val_acc: 0.7708
Epoch 6/100
29/29 [==============================] - 16s 564ms/step - loss: 0.2803 - acc: 0.8728 - val_loss: 0.4038 - val_acc: 0.7865
Epoch 7/100
29/29 [==============================] - 16s 556ms/step - loss: 0.2485 - acc: 0.8895 - val_loss: 0.4065 - val_acc: 0.8385
Epoch 8/100
29/29 [==============================] - 16s 557ms/step - loss: 0

Epoch 23/100
29/29 [==============================] - 15s 506ms/step - loss: 0.0612 - acc: 0.9779 - val_loss: 0.3522 - val_acc: 0.8646
Epoch 24/100
29/29 [==============================] - 14s 481ms/step - loss: 0.0778 - acc: 0.9720 - val_loss: 0.3904 - val_acc: 0.8438
Epoch 25/100
29/29 [==============================] - 15s 508ms/step - loss: 0.0647 - acc: 0.9752 - val_loss: 0.4134 - val_acc: 0.8438
Epoch 26/100
29/29 [==============================] - 15s 507ms/step - loss: 0.0598 - acc: 0.9774 - val_loss: 0.4021 - val_acc: 0.8646
Epoch 27/100
29/29 [==============================] - 14s 499ms/step - loss: 0.0581 - acc: 0.9774 - val_loss: 0.3592 - val_acc: 0.8646
Epoch 28/100
29/29 [==============================] - 14s 497ms/step - loss: 0.0549 - acc: 0.9811 - val_loss: 0.4018 - val_acc: 0.8594
Epoch 29/100
29/29 [==============================] - 15s 509ms/step - loss: 0.0669 - acc: 0.9736 - val_loss: 0.4520 - val_acc: 0.8542
--------------------------

AUC: 0.990453
ACC: 0.974138

In [7]:
import numpy as np 
results1 =   np.load( 'seq_and_go__incep_0.npz')
results2 =   np.load( 'seq_and_go__incep_1.npz')
results3 =   np.load( 'seq_and_go__incep_2.npz')
print ('mean AUC: %f' %  ( (np.mean( results1[ 'AUCs' ] )  + np.mean(  results2[ 'AUCs' ] )  + np.mean(results3[ 'AUCs' ]))/3     ) )
print ('mean ACC: %f' %   ( (np.mean( results1[ 'ACCs' ] )  + np.mean(  results2[ 'ACCs' ] )  + np.mean(results3[ 'ACCs' ]))/3) )
print ('mean MCC : %f' %  (  (np.mean( results1[ 'MCCs' ] )  + np.mean(  results2[ 'MCCs' ] )  + np.mean(results3[ 'MCCs' ])     )/3))
print('mean TPR:%f'%    ((np.mean( results1[ 'TPRs' ] )  + np.mean(  results2[ 'TPRs' ] )  + np.mean(results3[ 'TPRs' ])     )/3))
print('mean FPR:%f'%   ( (np.mean( results1[ 'FPRs' ] )  + np.mean(  results2[ 'FPRs' ] )  + np.mean(results3[ 'FPRs' ])     )/3))
print('mean Pre:%f'%    ((np.mean( results1[ 'Precs' ] )  + np.mean(  results2[ 'Precs' ] )  + np.mean(results3[ 'Precs' ])     )/3))
print('mean F1:%f'%    ((np.mean( results1[ 'F1s' ] )  + np.mean(  results2[ 'F1s' ] )  + np.mean(results3[ 'F1s' ])     )/3))

mean AUC: 0.974900
mean ACC: 0.925042
mean MCC : 0.854193
mean TPR:0.895629
mean FPR:0.045964
mean Pre:0.953164
mean F1:0.921222
